In [9]:
# Find the average rating of movie stars
# Find the average rating of each movie star along with their names and birthdays.
# Sort the result in the ascending order based on the birthday.
# Import your libraries
import pandas as pd
# import datatime
nominee_information=pd.read_csv('nominee_information.csv')
nominee_filmography=pd.read_csv('nominee_filmography.csv')
# Start writing code
df = nominee_filmography.groupby('name')['rating'].mean().reset_index()

result = pd.merge(df,nominee_information[['name','birthday']], on= 'name',how='inner')
result['birthday'] =  pd.to_datetime(result['birthday'], format='%d-%m-%Y' ).dt.strftime('%Y-%m-%d')
result.sort_values(by='birthday')

# select  distinct  f.name,birthday,avg(rating)over(partition by f.name)
# from nominee_filmography f
# join nominee_information i
# on  f.name=i.name
# order by 2

,name,rating,birthday
48,Cary Grant,NaN,1904-01-18
188,John Wayne,NaN,1907-05-26
20,Anna Magnani,NaN,1908-03-07
156,James Stewart,NaN,1908-05-20
214,Kevin McCarthy,NaN,1914-02-15
...,...,...,...
168,Jennifer Lawrence,6.400000,1990-08-15
332,Saoirse Ronan,5.000000,1994-04-12
0,Abigail Breslin,5.750000,1996-04-14
132,Hailee Steinfeld,4.666667,1996-12-11


In [11]:
# # Differences In Movie Ratings
# # Calculate the average lifetime rating and rating from the second latest film across all actors and all films they had acted in. 
# # Output a list of actors, their average lifetime rating, rating from the second latest film, and the difference between the two ratings.


import pandas as pd
import numpy as np
nominee_information=pd.read_csv('nominee_information.csv')
nominee_filmography=pd.read_csv('nominee_filmography.csv')
df = nominee_filmography[~nominee_filmography['rating'].isna()]
df['film_order'] = df.groupby(['name'])['id'].rank(ascending=True)
result = df[df['film_order']==2][['name', 'rating']].set_index('name')
result['lifetime_rating'] = df.groupby(['name'])['rating'].mean()
result['variance'] = result['rating'] - result['lifetime_rating']
result = result.reset_index()
result



# with a as
# (select  name,rating,id,row_number()over(partition by name order by name desc, id asc) rn
# from nominee_filmography
#  where rating is not null),
#  b as (select name,ROUND(AVG(rating)::numeric,1) as lifetime_rating	
#  from nominee_filmography 
# where rating is not null  group by 1)
# select a.name, a.rating second_last_rating ,lifetime_rating,
# round((a.rating -lifetime_rating)::numeric, 1)  variance	
#  from a join b
# on a.name=b.name
# where rn=2


2021-06-11 23:44:09,923 [4708] WARNING  py.warnings:110: [JupyterRequire] C:\Users\bluii\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()



,name,rating,lifetime_rating,variance
0,Helena Bonham Carter,2.0,4.000000,-2.000000
1,Jacki Weaver,5.0,6.250000,-1.250000
2,Anna Kendrick,4.0,4.571429,-0.571429
3,Pen_lope Cruz,5.0,3.500000,1.500000
4,Peter Fonda,2.0,2.000000,0.000000
...,...,...,...,...
201,Reese Witherspoon,6.0,6.500000,-0.500000
202,Sam Shepard,5.0,5.333333,-0.333333
203,Ellen Page,6.0,5.333333,0.666667
204,Robert De Niro,3.0,3.750000,-0.750000


In [24]:
# # Find the genre of the person with the most number of oscar winnings
# # Find the genre of the person with the most number of oscar winnings.
# # If there are more than one person with the same number of oscar wins, return the first one in alphabetic order.



import pandas as pd
import numpy as np
nominee_information=pd.read_csv('nominee_information.csv')
nominee_filmography=pd.read_csv('nominee_filmography.csv')
oscar_nominees=pd.read_csv('oscar_nominees.csv')
winner = oscar_nominees[oscar_nominees['winner'] == True]
n_winnings = winner.groupby('nominee').size().to_frame('n_win').reset_index()
merged = pd.merge(n_winnings, nominee_information, left_on = 'nominee', right_on = 'name', how = 'left')
result = merged.groupby(['top_genre']).apply(lambda x: x.nlargest(1, 'n_win')).reset_index(drop=True).sort_values('n_win',ascending = False).head(1)
print(result)
result[['top_genre']]




# with a as(select count(name),top_genre
# from nominee_information i
# join oscar_nominees n
# on i.name=n.nominee
# where winner='TRUE'
# group by 2
# order by 1 desc
# limit 1)
# select top_genre from a

          nominee  n_win            name amg_person_id top_genre    birthday  \
4  Ingrid Bergman      3  Ingrid Bergman        P 5652     Drama  29-08-1915   

    id  
4  140  


,top_genre
4,Drama


In [18]:
df = pd.DataFrame({'a':[0,0,1,2,2,2], 'b':[1,2,3,4,np.NaN,np.NaN,], 'c':np.random.randn(6)})
df
print('count',df.groupby(['a'])['b'].count())

print('size',df.groupby(['a'])['b'].size())


count a
0    2
1    1
2    1
Name: b, dtype: int64
size a
0    2
1    1
2    3
Name: b, dtype: int64


In [25]:
import pandas as pd

merged_df = pd.merge(oscar_nominees, nominee_information,  how = 'left',  left_on = 'nominee', right_on = 'name')

winner_df = merged_df[merged_df['winner'] == True]

result = winner_df.groupby(['nominee', 'top_genre'])['winner'] .count().rename('n_winnings').reset_index().sort_values(['n_winnings', 'nominee'], ascending = [False, True])

result['top_genre'].head(1)

110    Drama
Name: top_genre, dtype: object

In [30]:
# Import your libraries
import pandas as pd

# Start writing code
oscar_nominees.head()

df2= oscar_nominees[oscar_nominees["winner"]][["nominee","winner"]]
n_winnings = df2.groupby('nominee').size().to_frame('n_win').reset_index()
winner= n_winnings.sort_values(by=["n_win","nominee"],ascending=[False,True])[["nominee"]].values[0]

nominee_information[nominee_information["name"]==winner[0]]["top_genre"]

139    Drama
Name: top_genre, dtype: object